In [16]:
!pip install tensorflow==2.15

You should consider upgrading via the 'C:\Users\youssef\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [17]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Flatten, Dropout,BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow_hub as hub
from sklearn.metrics import confusion_matrix,classification_report,multilabel_confusion_matrix
import tqdm
from sklearn.metrics import cohen_kappa_score
import cv2
from tensorflow.image import resize
import re


In [18]:
print(tf.__version__)

2.15.0


In [19]:
NUM_BONE_TYPES = 12
IMAGE_SIZE = (256, 256)
batch_size = 32

In [20]:
class MyEfficientNetModel(tf.keras.Model):
    def __init__(self):
        super(MyEfficientNetModel, self).__init__()
        model_url = "https://tfhub.dev/google/efficientnet/b5/feature-vector/1"
        self.efficientnet_model = hub.KerasLayer(model_url, trainable=False)
        self.dense_layer6 = Dense(32,activation='relu')
        self.batchnorm6 = BatchNormalization()
        self.dropout_layer6 = Dropout(0.3)
        self.dense_layer5 = Dense(32,activation='relu')
        self.batchnorm5 = BatchNormalization()
        self.dropout_layer5 = Dropout(0.3)
        self.dense_layer4 = Dense(32,activation='relu')
        self.batchnorm4 = BatchNormalization()
        self.dropout_layer4 = Dropout(0.3)
        self.dense_layer3 = Dense(32,activation='relu')
        self.batchnorm3 = BatchNormalization()
        self.dropout_layer3 = Dropout(0.3)
        self.dense_layer2 = Dense(32,activation='relu')
        self.batchnorm2 = BatchNormalization()
        self.dropout_layer2 = Dropout(0.3)
        self.dense_layer = Dense(NUM_BONE_TYPES, activation='sigmoid')

    def call(self, inputs):
        x = self.efficientnet_model(inputs)
        dense6 = self.dense_layer6(x)
        batch6 = self.batchnorm6(dense6)
        dropout6 = self.dropout_layer6(batch6)
        dense5 = self.dense_layer5(dropout6)
        batch5 = self.batchnorm5(dense5)
        dropout5 = self.dropout_layer5(batch5)
        dense4 = self.dense_layer4(dropout5)
        batch4 = self.batchnorm4(dense4)
        dropout4 = self.dropout_layer4(batch4)
        dense3 = self.dense_layer3(dropout4)
        batch3 = self.batchnorm3(dense3)
        dropout3 = self.dropout_layer3(batch3)
        dense2 = self.dense_layer2(dropout3)
        batch2 = self.batchnorm2(dense2)
        dropout2 = self.dropout_layer2(batch2)
        outputs = self.dense_layer(dropout2)
        return outputs

model = MyEfficientNetModel()

dummy_input = np.zeros((1, *IMAGE_SIZE, 3), dtype=np.float32)
_ = model(dummy_input)

model.load_weights(r"C:\Users\youssef\Desktop\API\EfficientnetMulti_label_optimiz.h5")

In [21]:
def Testing(Imagepath, mymodel):
    test_image = cv2.imread(Imagepath,cv2.IMREAD_COLOR)
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)  
    lab = cv2.cvtColor(test_image, cv2.COLOR_RGB2Lab)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)
    lab_clahe = cv2.merge([l_clahe, a, b])
    clahe_image = cv2.cvtColor(lab_clahe, cv2.COLOR_Lab2RGB)
    clahe_image=cv2.cvtColor(clahe_image, cv2.COLOR_RGB2BGR)
    test_image = clahe_image / 255.0
    test_image = resize(clahe_image, (256, 256))
    test_image = np.expand_dims(test_image, axis=0)  

    probability_test = mymodel.predict(test_image)

    classes = ['ANKLE', 'ELBOW', 'FINGER', 'FOOT', 'FOREARM', 'HAND', 'HIP', 'HUMERUS', 'KNEE', 'SHOULDER', 'WRIST', 'Abnormalitiy']
    sorted_indices = np.argsort(probability_test[0])[::-1]

    Predicted_Classes = {}
    for i in range(12):
        Predicted_Classes[classes[sorted_indices[i]]] = probability_test[0][sorted_indices[i]]

    top_class = None
    for idx in sorted_indices:
        if classes[idx] != 'Abnormalitiy':
            top_class = classes[idx]
            break

    print("The x-Ray Bone type is", top_class)
    Fractured = False
    abnormality_probability = probability_test[0, -1] 
    if abnormality_probability >= 0.35:
        print("Positive: Fractured Bone")
        Fractured = True
        explainer = lime_image.LimeImageExplainer()
        input_image = test_image[0]  

        explanation = explainer.explain_instance(
        input_image,
        mymodel.predict,
        top_labels=2,  
        hide_color=0,
        num_samples=1000 
        )

        temp, mask = explanation.get_image_and_mask(
            11,
            positive_only=True,
            num_features=5,  
            hide_rest=False,
            min_weight=0.0
        )

        plt.imshow(input_image)
        plt.imshow(mask, cmap='jet', alpha=0.2) 
        plt.axis('off')
        plt.show()
    else:
        print("Negative: Normal Bone")

    return Predicted_Classes,Fractured
